# Working with IBL atlas object

## Getting started

In [ ]:
from ibllib.atlas import AllenAtlas

res = 25 # resolution of Atlas, available resolutions are 10, 25 (default) and 50
brain_atlas = AllenAtlas(res_um=res)

## Exploring the volumes

### 1. Image Volume 
Allen atlas dwi average template

In [ ]:
# Access the image volume
im = brain_atlas.image

# Explore the size of the image volume (ap, ml, dv)
im.shape

# Plot a coronal slice at ap = -1000um
ap = -1000 / 1e6  # input must be in metres
ax = brain_atlas.plot_cslice(ap, volume='image')


### Label Volume


In [ ]:
brain_atlas.bc.nxyz

In [ ]:
brain_atlas.regions.acronym.shape

In [ ]:
brain_atlas.bc.xyz2i([-1000/1e6, -1000/1e6, -4000/1e6])

In [ ]:
brain_atlas.image[190, 256, 173] = 100000
brain_atlas.image[256, 190, 173] = 100000000

In [ ]:
ax = brain_atlas.plot_cslice(ap, volume='image', vmax=10000)

In [ ]:
# understainding